In [ ]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [ ]:
df = pd.read_csv("../data/adult.csv")
df = df.replace("?", np.nan)
df = df.dropna()
df = df[df["salary"].isin(["<=50K", ">50K"])]
df["salary K$"] = pd.to_numeric(df["salary K$"], errors="coerce")
df = df.dropna(subset=["salary K$"])
df

In [1]:
gender_counts = df["sex"].value_counts()
print("Кількість чоловіків і жінок у наборі даних:")
print(gender_counts)

# Task 2
Find the average age of men in dataset

In [ ]:
average_age_men = df[df["sex"] == "Male"]["age"].mean()
print("Середній вік чоловіків у наборі даних:", round(average_age_men, 2))

# Task 3
Get the percentage of people from Poland (native-country)

In [ ]:
total_people = len(df)
poland_count = len(df[df["native-country"] == "Poland"])
poland_percentage = (poland_count / total_people) * 100
print(f"Відсоток людей з Польщі: {poland_percentage:.2f}%")

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [2]:
high_income = df[df["salary"] == ">50K"]["age"]
mean_high_income = high_income.mean()
std_high_income = high_income.std()

low_income = df[df["salary"] == "<=50K"]["age"]
mean_low_income = low_income.mean()
std_low_income = low_income.std()

print(f"Середній вік для тих, хто заробляє >50K: {mean_high_income:.2f}")
print(f"Стандартне відхилення віку для тих, хто заробляє >50K: {std_high_income:.2f}")
print(f"Середній вік для тих, хто заробляє <=50K: {mean_low_income:.2f}")
print(f"Стандартне відхилення віку для тих, хто заробляє <=50K: {std_low_income:.2f}")

# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [3]:
higher_education = ["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"]

non_higher_educated_high_income = df[(df["salary"] == ">50K") & (~df["education"].isin(higher_education))]

print("Люди без вищої освіти, але з зарплатою понад 50K:")
non_higher_educated_high_income

# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [ ]:
age_statistics_by_education = df.groupby("education")["age"].describe()

print("Статистика віку для кожного типу освіти:")
print(age_statistics_by_education)

# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [5]:
married_people = df[df["marital-status"].str.startswith("Married")]
non_married_people = df[~df["marital-status"].str.startswith("Married")]

married_high_income = married_people[married_people["salary"] == ">50K"].shape[0] / married_people.shape[0] * 100
non_married_high_income = non_married_people[non_married_people["salary"] == ">50K"].shape[0] / non_married_people.shape[0] * 100

print("Відсоток одружених людей з доходом більше 50K:", round(married_high_income, 2), "%")
print("Відсоток неодружених людей з доходом більше 50K:", round(non_married_high_income, 2), "%")

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [6]:
max_hours = df["hours-per-week"].max()

people_with_max_hours = df[df["hours-per-week"] == max_hours].shape[0]

print("Максимальна кількість годин на тиждень:", max_hours)
print("Кількість людей, які працюють стільки годин:", people_with_max_hours)

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [7]:
correlation_matrix = df.select_dtypes(include=["number"]).corr()
print(correlation_matrix)